In [ ]:
import os
import shutil
import json

# 원본 데이터 폴더와 생성할 학습 데이터셋 폴더 경로 설정
source_dir = 'train_landscape_person'
dest_dir = 'train_dataset'

# 1. ./train_dataset/ 폴더 만들기
# exist_ok=True 옵션은 폴더가 이미 존재할 경우 오류를 발생시키지 않습니다.
print(f"'{dest_dir}' 폴더를 생성합니다...")
os.makedirs(dest_dir, exist_ok=True)

# 메타데이터를 저장할 리스트 초기화
metadata = []
copied_files_count = 0

print(f"'{source_dir}' 폴더에서 데이터 처리를 시작합니다...")

# 2. 원본 데이터 폴더 내 파일들을 순회
try:
    file_list = os.listdir(source_dir)
except FileNotFoundError:
    print(f"[오류] 원본 폴더 '{source_dir}'를 찾을 수 없습니다. 스크립트와 같은 위치에 폴더가 있는지 확인하세요.")
    exit()

for filename in file_list:
    # 3. .txt 확장자를 가진 파일을 기준으로 작업 수행
    if filename.endswith('.txt'):
        base_name = os.path.splitext(filename)[0]
        
        txt_path = os.path.join(source_dir, filename)
        jpg_filename = f"{base_name}.jpg" # 또는 .png 등 다른 이미지 확장자도 고려한다면,
                                          # 여기에 추가적인 로직을 넣거나, 아래 jpg_path 확인 시 확장자 무관하게 확인 가능
        jpg_path = os.path.join(source_dir, jpg_filename)
        
        # 4. .txt와 이름이 같은 .jpg 파일이 있는지 확인
        if os.path.exists(jpg_path):
            # .txt 파일에서 캡션 읽기 (UTF-8 인코딩 사용)
            try:
                with open(txt_path, 'r', encoding='utf-8') as f:
                    caption = f.read().strip()
            except Exception as e:
                print(f"'{txt_path}' 파일 읽기 오류: {e}")
                continue

            # .jpg 파일을 ./train_dataset/ 폴더로 복사 (shutil.copy 사용)
            dest_jpg_path = os.path.join(dest_dir, jpg_filename)
            shutil.copy(jpg_path, dest_jpg_path) # <-- 이 부분이 shutil.move에서 shutil.copy로 변경되었습니다.
            
            # 5. SDXL 형식에 맞는 메타데이터 생성
            metadata_entry = {
                "file_name": jpg_filename,
                "text": caption
            }
            metadata.append(metadata_entry)
            copied_files_count += 1
            
            # 처리 중인 파일 정보 출력
            print(f" - 처리 완료: {jpg_filename} -> 캡션: \"{caption[:30]}...\"")

# 6. ./train_dataset/metadata.jsonl 파일로 저장
metadata_path = os.path.join(dest_dir, 'metadata.jsonl')
try:
    with open(metadata_path, 'w', encoding='utf-8') as f:
        for entry in metadata:
            # ensure_ascii=False 옵션은 한글이 깨지지 않도록 합니다.
            f.write(json.dumps(entry, ensure_ascii=False) + '\n')
except Exception as e:
    print(f"'{metadata_path}' 파일 쓰기 오류: {e}")


print("\n--- 작업 완료 ---")
print(f"총 {copied_files_count}개의 .jpg 파일이 '{dest_dir}'로 복사되었습니다.")
print(f"'{metadata_path}' 파일이 생성되었습니다.")

In [ ]:
!accelerate launch train_text_to_image_lora_sdxl.py \
  --pretrained_model_name_or_path="stabilityai/stable-diffusion-xl-base-1.0" \
  --train_data_dir="./train_dataset/" \
  --caption_column="text" \
  --resolution=1024 \
  --random_flip \
  --train_batch_size=1 \
  --num_train_epochs=10 \
  --checkpointing_steps=240 \
  --learning_rate=1e-04 \
  --lr_scheduler="cosine" \
  --lr_warmup_steps=0 \
  --mixed_precision="fp16" \
  --seed=42 \
  --output_dir="sdxl-lora-minhwa-style" \
  --validation_prompt="Korean traditional minhwa painting, two person, a men wearing Korean hat, gat, blue clothes, a men wearing blue clothes, with a background of green mountain landscape." \
  --report_to="wandb" \
  --rank=16